In [1]:
!pip install opencv-contrib-python tqdm matplotlib numpy ipywidgets


  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/81/3c/bbb3ceee9fbefc505f98c24dafda68c7b3c4f83b6951c0712b4623fe4cce/opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl (44.8 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\eaell\\Anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [23]:
# Define the size of the Charuco board (number of squares along the width and height)
board_width = 6
board_height = 4
square_size = 0.025  # Size of each square in meters

# Generate Charuco board
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
board = cv2.aruco.CharucoBoard([board_width, board_height], 0.025, 0.0125, dictionary)
board_image = board.generateImage((100 * board_width, 100 * board_height))
show_frame(board_image)
cv2.imwrite('board.jpg',board_image)
# Create a list to store Charuco board images and corresponding object points
all_charuco_corners = []
all_charuco_ids = []
all_object_points = []


# Capture video from the camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    clear_output(wait=True)

    # Detect Charuco board corners and ids
    detector_params = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, detector_params)
    corners, ids, _ = detector.detectMarkers(frame)
    chardetect = cv2.aruco.CharucoDetector(board,cv2.aruco.CharucoParameters(),detector_params)
    charuco_corners, charuco_ids, _, _ = chardetect.detectBoard(frame)

    # If Charuco board is detected, display it
    if charuco_corners is not None and charuco_ids is not None:
        #cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        cv2.aruco.drawDetectedCornersCharuco(frame, charuco_corners, charuco_ids)

        # Store Charuco board corners, ids, and object points
        all_charuco_corners = charuco_corners
        all_charuco_ids = charuco_ids
        all_object_points = board.getChessboardCorners()
        print(all_object_points)
        print(all_charuco_corners)
        # Create a list to store object points (3D points in real-world coordinates)

        if len(all_charuco_corners) > 0:
            ret, camera_matrix, dist_coeffs, _, _ = cv2.calibrateCamera(all_object_points, all_charuco_corners, frame.shape[1:], None, None)

    show_frame(frame)

# Release the camera
cap.release()
cv2.destroyAllWindows()

# Perform camera calibration


IndentationError: expected an indented block after 'if' statement on line 47 (2228285499.py, line 50)

In [2]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as col
import IPython
from IPython.display import display, clear_output
import time
import math
from cvision2 import *
from Global_NAV import *

# Create an adjusted dictionary of 5 ArUco markers of 4x4 bits and a detector of these markers.
aruco_dictionary = cv2.aruco.extendDictionary(5, 4)
detector = create_detector(aruco_dictionary)
# Print a marker for each objects to be recognized on the map
for i in range(5):
    marker = cv2.aruco.generateImageMarker(aruco_dictionary, i, 500, np.zeros((6, 6, 1), dtype="uint8"), 1)
    cv2.imwrite('marker' + str(i) +'.jpg', marker)
# Definition of the references used to map the space
ref1 = Obj(0, np.array([[0,0]]))
ref2 = Obj(1, np.array([[0,0]]))
ref3 = Obj(2, np.array([[0,0]]))

# Definition of the objects to recognize in the space
goal = Obj(3, np.array([[]]))
thymio = Obj(4, np.array([[]]))

map = Map(ref1,ref2,ref3,640,640)

objects = np.array([ref1,ref2, ref3, goal, thymio])
                
# Capture video from your camera or load an image
cap = cv2.VideoCapture(0)  # Change to the camera index or video file path if using video

#for i in range(3):
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    clear_output(wait=True)
    #show_frame(frame)
    
    
    detect_objects(frame, detector, objects)
    
    if map.references_detected():
        matrix = map.matrix(frame,thymio,goal)
        if np.size(map.localize(thymio), 0) > 0:
            print('Thymio map pos', map.localize(thymio))
            print('Thymio map dir', vector_to_angle(map.vec_img2map(thymio.get_img_dir()[0])))
        if check_matrix(matrix):
            path,visitedNodes = global_path(matrix)
            print_path(matrix,path,visitedNodes)
            path = np.multiply(path, 100)
            N = np.size(path,1)
            print("N" ,N)
    
    # Draw the image with the recognized objects
    new_frame = frame.copy()
    ref1.draw(new_frame, [255,0,0])
    ref2.draw(new_frame, [0,255,0])
    ref3.draw(new_frame, [0,0,255])
    goal.draw(new_frame, [255,255,255])
    thymio.draw(new_frame, [255,100,100])
    show_frame(new_frame)

# Release the video capture object
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 